# Customer Churn Prediction using PyTorch

In [1]:
# install PyTorch
!pip install torch torchvision

In [2]:
#import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# Load the dataset
df = pd.read_csv("Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# Drop customerID and handle missing values
df.drop("customerID", axis=1, inplace=True)
df.replace(" ", np.nan, inplace=True)
df.dropna(inplace=True)

In [5]:
# Convert TotalCharges to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])

In [6]:
print(df["Churn"].unique())

['No' 'Yes']


In [7]:
# Encode target
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [8]:
# Encode categorical features
cat_cols = df.select_dtypes(include=["object"]).columns
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


In [9]:
# Split features and target
X = df.drop("Churn", axis=1).values
y = df["Churn"].values

In [10]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-1.00943013, -0.44032709,  1.03561683, ...,  0.39980518,
        -1.16169394, -0.99419409],
       [ 0.99065797, -0.44032709, -0.9656081 , ...,  1.33663626,
        -0.26087792, -0.17373982],
       [ 0.99065797, -0.44032709, -0.9656081 , ...,  1.33663626,
        -0.36392329, -0.95964911],
       ...,
       [-1.00943013, -0.44032709,  1.03561683, ...,  0.39980518,
        -1.17000405, -0.85451414],
       [ 0.99065797,  2.27103902,  1.03561683, ...,  1.33663626,
         0.31916782, -0.87209546],
       [ 0.99065797, -0.44032709, -0.9656081 , ..., -1.47385696,
         1.35793167,  2.01234407]])

In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [13]:
# Create datasets and loaders
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

In [14]:
# Define neural network
class ChurnModel(nn.Module):
    def __init__(self, input_dim):
        super(ChurnModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),   # Layer 1: Input → 64 neurons
            nn.ReLU(),                  # Activation
            nn.Linear(64, 32),          # Layer 2: 64 → 32 neurons
            nn.ReLU(),                  # Activation
            nn.Linear(32, 16),          # Layer 3: 32 → 16 neurons
            nn.ReLU(),                  # Activation
            nn.Linear(16, 1),           # Final layer: 16 → 1 output neuron
            nn.Sigmoid()                # Activation to get probability (0–1)
        )

    def forward(self, x):
        return self.net(x)

In [15]:
# Initialize model
model = ChurnModel(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/50, Loss: 0.4881
Epoch 2/50, Loss: 0.4230
Epoch 3/50, Loss: 0.4147
Epoch 4/50, Loss: 0.4111
Epoch 5/50, Loss: 0.4077
Epoch 6/50, Loss: 0.4044
Epoch 7/50, Loss: 0.4004
Epoch 8/50, Loss: 0.3989
Epoch 9/50, Loss: 0.3953
Epoch 10/50, Loss: 0.3929
Epoch 11/50, Loss: 0.3916
Epoch 12/50, Loss: 0.3870
Epoch 13/50, Loss: 0.3870
Epoch 14/50, Loss: 0.3816
Epoch 15/50, Loss: 0.3795
Epoch 16/50, Loss: 0.3786
Epoch 17/50, Loss: 0.3746
Epoch 18/50, Loss: 0.3704
Epoch 19/50, Loss: 0.3674
Epoch 20/50, Loss: 0.3647
Epoch 21/50, Loss: 0.3613
Epoch 22/50, Loss: 0.3587
Epoch 23/50, Loss: 0.3558
Epoch 24/50, Loss: 0.3517
Epoch 25/50, Loss: 0.3486
Epoch 26/50, Loss: 0.3479
Epoch 27/50, Loss: 0.3424
Epoch 28/50, Loss: 0.3392
Epoch 29/50, Loss: 0.3360
Epoch 30/50, Loss: 0.3343
Epoch 31/50, Loss: 0.3293
Epoch 32/50, Loss: 0.3276
Epoch 33/50, Loss: 0.3260
Epoch 34/50, Loss: 0.3206
Epoch 35/50, Loss: 0.3188
Epoch 36/50, Loss: 0.3149
Epoch 37/50, Loss: 0.3129
Epoch 38/50, Loss: 0.3078
Epoch 39/50, Loss: 0.

In [17]:
# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = (predictions > 0.5).float()
    acc = accuracy_score(y_test_tensor, predicted_classes)
    print(f"Test Accuracy: {acc * 100:.2f}%")
    print("Classification Report:")
    print(classification_report(y_test_tensor, predicted_classes))

Test Accuracy: 75.69%
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.84      0.84      1033
         1.0       0.54      0.52      0.53       374

    accuracy                           0.76      1407
   macro avg       0.69      0.68      0.68      1407
weighted avg       0.75      0.76      0.76      1407



In [18]:
# Save the model
torch.save(model.state_dict(), "churn_model.pth")
print("Model saved to churn_model.pth")

Model saved to churn_model.pth
